# Example of fixed-base and chained bilateral indices (from Aizcorbe 2014, example 2.1)

In Example 2.1, Aizcorbe uses US National Income and Product Accounts data from the BEA. This data is provided with the repo. Instead of the raw format on the BEA website and the massaged format on Table 2.3, it is instead provided in two separate `.csv` files

In [2]:
import pandas as pd

In [3]:
df_expenditures = pd.read_csv("../data/silver/2000-2010 US NIPA data - nominal spending.csv")
df_expenditures

,Years,Motor Vehicles and Parts,Furnishings and Durable Household Equipment,Recreational Goods and Vehicles,Other Durable Goods
0,2000,363.2,208.1,234.1,110.4
1,2001,383.3,214.9,239.8,108.4
2,2002,401.3,225.9,251.5,113.4
3,2003,401.0,231.8,265.7,121.4
4,2004,403.9,247.0,290.5,131.5
5,2005,408.2,261.3,312.8,141.1
6,2006,394.8,271.5,334.1,154.6
7,2007,399.9,271.3,349.4,167.8
8,2008,339.3,257.9,344.0,167.7
9,2009,316.5,235.3,316.6,161.2


In [4]:
df_price_indices = pd.read_csv("../data/silver/2000-2010 US NIPA data - price indexes.csv")
df_price_indices

,Years,Motor Vehicles and Parts,Furnishings and Durable Household Equipment,Recreational Goods and Vehicles,Other Durable Goods
0,2000,102.00,108.12,136.29,105.23
1,2001,102.40,106.27,126.99,105.56
2,2002,101.86,104.08,118.78,103.61
3,2003,99.08,101.16,111.28,101.91
4,2004,98.40,99.91,105.76,101.54
5,2005,100.00,100.00,100.00,100.00
6,2006,100.11,99.60,93.79,101.80
7,2007,99.62,98.89,87.16,105.53
8,2008,97.82,98.14,82.65,109.09
9,2009,98.16,97.82,77.35,110.40
